<a href="https://colab.research.google.com/github/darischen/CSE110Pages/blob/main/Neural_Network_Beam_Profiling_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Detectron2 to profile laser beams

<img src="https://github.com/Dipolar-Quantum-Gases/nn-beam-profiling/blob/master/imgs/thumbnail.jpg?raw=true" alt="drawing" width="150"/> <img src="https://github.com/Dipolar-Quantum-Gases/nn-beam-profiling/blob/master/imgs/thumbnailexp.png?raw=true" alt="drawing" width="150"/>

This notebook demonstrates how to train a deep neural network to detect and profile multiple laser beams in a single image. This is based off the paper "Measuring Laser Beams with a Neural Network" which can be found in both [ArXiv](https://arxiv.org/abs/2202.07801) and [published versions](https://doi.org/10.1364/AO.443531).

##Package Installation

### Install Detectron2

Detectron2 installation needs to match the PyTorch and CUDA versions on Colab's virtual machine.

In [6]:
!pip install detectron2 # -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html

ERROR: Invalid requirement: 'detectron2#': Expected end or semicolon (after name and no valid version specifier)
    detectron2#
              ^


###Clone Github repository
Let's clone the Github repo so we can use the code in the Colab notebook.

In [4]:
!git clone https://github.com/Dipolar-Quantum-Gases/nn-beam-profiling

Cloning into 'nn-beam-profiling'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 40 (delta 11), reused 30 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 1.25 MiB | 6.90 MiB/s, done.
Resolving deltas: 100% (11/11), done.


##Laser beam datasets

Their are two datasets used in the paper. The first is a simulated dataset comprised of 5000 images and the second is an experimental dataset with 1050 images created using a spatial light modulator. These are both located in an [University of Oxford Research Archive](https://ora.ox.ac.uk/objects/uuid:e7f9ee4c-5b07-469f-979c-73b8a28d7ec2) from which they can be downloaded as .zip files.

For more information on the dataset, we have a [Colab notebook](https://colab.research.google.com/github/Dipolar-Quantum-Gases/nn-beam-profiling/blob/master/Explore_the_Dataset.ipynb) which shows to inspect and visualize the two datasets

###Download the dataset

Here we download and unzip either the experimental or simulation dataset. Note the simulation dataset is 10x the size of the experimental dataset and can take a significant amount of time to download.

In [5]:
from pathlib import Path
import sys
sys.path.append('/content/nn-beam-profiling/nn-beam-profiling')
from dataset_extraction import dataset_extractor

#the .zip urls of the two datasets
dataset_urls = {'experimental_data': "https://ora.ox.ac.uk/objects/uuid:e7f9ee4c-5b07-469f-979c-73b8a28d7ec2/download_file?safe_filename=experimental_data.zip&type_of_work=Dataset",
                'simulation_data': "https://ora.ox.ac.uk/objects/uuid:e7f9ee4c-5b07-469f-979c-73b8a28d7ec2/download_file?safe_filename=simulation_data.zip&type_of_work=Dataset"}

zip_dir = Path('/content/data/datasets/compressed_data')
unzip_dir = Path('/content/data/datasets')
dataset = 'experimental_data'
# dataset = 'simulation_data'

url = dataset_urls[dataset]
dataset_extractor(dataset, url, zip_dir, unzip_dir)

ModuleNotFoundError: No module named 'dataset_extraction'

Here we get the paths to the different parts of the dataset including the annotations (text_path), the rgb images (img_path) and the top level directory (dataset_path).

In [ ]:
def get_paths(dataset, local_data_path):
  dataset_path = local_data_path /  dataset
  text_path = dataset_path / 'text'
  img_path = dataset_path / 'imgs'
  return dataset_path, text_path, img_path

dataset_path, text_path, img_path = get_paths(dataset, unzip_dir)

###Create Detectron2 dataset

Now we define a dataset class for our custom laser beam datasets. The dataframe file holds all the annotation information including boxes, rotated boxes, masks and keypoints. However here we focus on the rotated boxes.

In [ ]:
import detectron2
import pandas as pd
from detectron2.structures import BoxMode

class BeamDataset():

    def __init__(self, dataset_path, text_path, scalar=1):
        self.dataset_path = dataset_path
        self.text_path = text_path
        self.box_name = 'abox_' + str(scalar)


    def training_dataset(self):
        return self.get_dataset_list()


    def validation_dataset(self):
        return self.get_dataset_list(False)


    def read_DF(self, train):
        if train:
          self.data = pd.read_json(str(self.text_path) + '/data_train.json')
        else:
          self.data = pd.read_json(str(self.text_path) + '/data_val.json')


    def get_dataset_list(self, train=True):
        self.read_DF(train)
        # dlist = []
        dlist = [self.getitem(i, train) for i in range(len(self.data))]
        # length = len(self.data)
        # for i in range(0, length):
        #   dlist.append(self.getitem(i, train))
        return dlist


    def dataset_length(self, train=True):
        self.read_DF(train)
        return len(self.data)


    def getitem(self, idx, train=True):
        # load images and ROI boxes
        idata = self.data.loc[idx]
        aboxes = idata[self.box_name]
        labels = idata['label']
        annotations = []
        for abox, label in zip(aboxes, labels): # get bounding box coordinates and fits for each mask
            annotation_dict = {}
            annotation_dict['bbox'] = [float(val) for val in abox]
            annotation_dict['bbox_mode'] = BoxMode.XYWHA_ABS
            annotation_dict['category_id'] = label - 1
            annotation_dict['iscrowd'] = 0
            annotations.append(annotation_dict)

        image_dict = {}
        image_dict['annotations'] = annotations
        file_name = str(self.dataset_path) + idata['rgb_paths'].replace('\\', '/')
        image_dict['file_name'] = file_name
        image_dict['image_id'] = int(idata['run'])
        image_dict['height'] = idata['height']
        image_dict['width'] = idata['width']

        return image_dict

### Registering the dataset

Detectron2 requires us to register our datasets and their metadata before we can use them. This is done for both the training and validation datsets in the code below.

In [ ]:
from detectron2.data import DatasetCatalog
from detectron2.data import MetadataCatalog

def register_dataset_metadata(dataset_name, dataset_function, class_names):
  if dataset_name in DatasetCatalog.list():
    DatasetCatalog.remove(dataset_name)
    MetadataCatalog.remove(dataset_name)
  DatasetCatalog.register(dataset_name, dataset_function)
  MetadataCatalog.get(dataset_name).thing_classes = class_names


train_dataset = 'train_dataset'
val_dataset = 'val_dataset'
dataset_scalar = 1.5
class_names = ["Gaussian"]
acd = BeamDataset(dataset_path, text_path, scalar=dataset_scalar)
register_dataset_metadata(train_dataset, acd.training_dataset, class_names)
register_dataset_metadata(val_dataset, acd.validation_dataset, class_names)

###Visualize the dataset

Here we randomly visualize some of the validation dataset images along with their annotations for rotated regions-of-interest.



In [ ]:
import random
import cv2
from detectron2.utils.visualizer import Visualizer
from google.colab.patches import cv2_imshow


def get_dataset_info(dataset_name):
  dataset_metadata = MetadataCatalog.get(dataset_name)
  dataset_dicts = DatasetCatalog.get(dataset_name)
  return dataset_metadata, dataset_dicts


def visualize_dataset(dataset_metadata, dataset_dicts, num_samples=5, scale = 2):
  dlength = len(dataset_dicts)
  if num_samples > dlength:
    num_samples = dlength
  for d in random.sample(dataset_dicts, num_samples):
      img = cv2.imread(d["file_name"])
      visualizer = Visualizer(img[:, :, ::-1], metadata=dataset_metadata, scale=scale)
      vis = visualizer.draw_dataset_dict(d)
      cv2_imshow(vis.get_image()[:, :, ::-1])


training_dataset_metadata, training_dataset_dicts = get_dataset_info(train_dataset)
visualize_dataset(training_dataset_metadata, training_dataset_dicts, num_samples=2)

val_dataset_metadata, val_dataset_dicts = get_dataset_info(val_dataset)
visualize_dataset(val_dataset_metadata, val_dataset_dicts, num_samples=2)

##Neural network model

Rather than building the model explicitly in PyTorch, we modify a config file (CFG) to define the neural network we want built. Once we enter the training loop, Detectron2 will build the neural network model according to the CFG specifications.

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()

# get_base_model_and_weights
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# setup_rpn_head
cfg.MODEL.PROPOSAL_GENERATOR.NAME = "RRPN"
cfg.MODEL.RPN.HEAD_NAME = "StandardRPNHead"
cfg.MODEL.RPN.BBOX_REG_WEIGHTS = (1, 1, 1, 1, 1)

# setup_anchor_generator
cfg.MODEL.ANCHOR_GENERATOR.NAME = "RotatedAnchorGenerator"
cfg.MODEL.ANCHOR_GENERATOR.ANGLES = [[-60,-30,0,30,60,90]]

# Setup RROI heads
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
cfg.MODEL.ROI_HEADS.NAME = "RROIHeads"
cfg.MODEL.ROI_BOX_HEAD.POOLER_TYPE = "ROIAlignRotated"
cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_WEIGHTS = (1, 1, 1, 1, 1)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256

cfg.MODEL.MASK_ON = False
cfg.DATALOADER.NUM_WORKERS = 2

##Train

###Custom Detectron2 trainer

The default Detectron2 trainer doesn't work for rotated regions-of-interest so we write a custom trainer. This trainer needs a custom data mapper to be implemented in both the training and test loader.

Additionally, the evaluation between training epochs must be done with the Rotated COCO evaluator.

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.data import transforms as T
from detectron2.evaluation import DatasetEvaluators, RotatedCOCOEvaluator
from rotation_mapper import MyDatasetMapper


class MyTrainer(DefaultTrainer):
  def __init__(self, cfg):
    super().__init__(cfg)


  @classmethod
  def build_evaluator(self, cfg, dataset_name, output_folder=None):
      if output_folder is None:
          output_folder = cfg.OUTPUT_DIR + '/coco_eval/' + dataset_name
          os.makedirs(output_folder, exist_ok=True)
      evaluators = [RotatedCOCOEvaluator(dataset_name, cfg, True, output_folder)]
      return DatasetEvaluators(evaluators)


  @classmethod
  def build_train_loader(cls, cfg, hf=False, height=800, width=800):
    augmentations = T.AugmentationList([T.Resize((height, width))])
    return build_detection_train_loader(cfg, mapper=MyDatasetMapper(cfg, True,
                                                                    augmentations))


  @classmethod
  def build_test_loader(cls, cfg, dataset, height=800, width=800):
    augmentations = T.AugmentationList([T.Resize((height, width))])
    return build_detection_test_loader(cfg, dataset, mapper=MyDatasetMapper(cfg,
                                                                            False,
                                                                            augmentations))

###Set training parameters

Here we set the hyperparameters for training the neural network including batch size, learning rate, learning rate scheduler, momentum and training epochs.

Detectron2 uses iterations (number of batches passed through the NN) rather than training epochs so we need to convert between the two.

In [ ]:
from math import ceil

def get_learning_rate_schedule(lrs_iters, train_iters):
    max_steps = int(ceil(train_iters / lrs_iters))
    schedule = [lrs_iters * (i + 1) - 1 for i in range(0, max_steps)]
    return schedule


def epochs_to_iterations(epochs, num_imgs, batch_size):
    return int(epochs * (num_imgs / batch_size))

# define the parameters for training
train_epochs = 2
eval_epochs = 1
batch_size = 4
lrs_step = 10
num_imgs = len(DatasetCatalog.get(train_dataset))

# convert epochs to iterations
train_iters = epochs_to_iterations(train_epochs, num_imgs, batch_size)
eval_iters = epochs_to_iterations(eval_epochs, num_imgs, batch_size)
lrs_iters = epochs_to_iterations(lrs_step, num_imgs, batch_size)
lrs_schedule = get_learning_rate_schedule(lrs_iters, train_iters)

# set training hyperparameters in the config file
cfg.SOLVER.IMS_PER_BATCH = batch_size
cfg.SOLVER.BASE_LR = 0.01
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.GAMMA = .01
cfg.SOLVER.STEPS = lrs_schedule
cfg.SOLVER.WARMUP_ITERS = 0
cfg.SOLVER.MAX_ITER = train_iters
cfg.TEST.EVAL_PERIOD = eval_iters

# set training and validation datasets
cfg.DATASETS.TRAIN = (train_dataset,)
cfg.DATASETS.TEST = (val_dataset,)

###Train the neural network

Now we create folder for all the training/evaluation results and start the training loop. The custom trainer we defined earlier will construct the neural network according to the CFG file and then start the training loop with the hyperparameters also defined in the CFG file.

In [ ]:
import os
import shutil
import time

def make_output_dir(dataset, base_dir):
  output_dir = base_dir + '/' + dataset + '/' + time.strftime('%H''%M''_''%d''%m''%Y')
  cfg.OUTPUT_DIR = output_dir
  if os.path.isdir(output_dir):
    shutil.rmtree(output_dir)
  os.makedirs(output_dir, exist_ok=True)

make_output_dir(dataset, '/content/training_output')
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

##Evaluate

###Get the training/evaluation results

Let's first get the metrics file that Detectron saves throughout training. Detectron 2 spits out a training/validation metrics .json file. This is kind of a jumble and here we parse it to get out a training dataframe and an evaluation dataframe.

In [ ]:
def get_sub_df(metricsDF, split_string):
  sub_df = metricsDF.dropna(subset=[split_string]) #split validation metric
  sub_df = sub_df.dropna(axis=1, how='any') #drop columns with na values
  sub_df = sub_df.sort_values(by=['iteration']) #sort by training iteration
  sub_df = sub_df.reset_index(drop=True)
  return sub_df


def get_metrics(metrics_path):
  '''Does all the splitting of the metrics df into its respective components.'''
  metricsDF = pd.read_json(metrics_path + '/metrics.json', lines=True)
  training_df = get_sub_df(metricsDF, 'total_loss')
  evaluation_df = get_sub_df(metricsDF, 'bbox/APs')
  return training_df, evaluation_df

training_df, evaluation_df = get_metrics(cfg.OUTPUT_DIR)
evaluation_df

###Plot the evaluation results

We plot the mAP after each training epoch and the loss during training.

In [ ]:
evaluation_df.plot('iteration', 'bbox/AP')
training_df.plot('iteration', 'total_loss')

##Using the trained model

###Inference

We want to use the trained neural network on images which it was not trained on. Below we create an inference predictor with weights from the trained model.

We then randomly select images from the validation dataset and perform inference on them.

In [ ]:
from detectron2.engine import DefaultPredictor

def pred_abox_list(outputs):
  tensor_pred = outputs['instances'].get_fields()['pred_boxes'].tensor
  pred_list = tensor_pred.cpu().numpy().tolist()
  return pred_list

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST =0.8
predictor = DefaultPredictor(cfg)

num_samples = 5
for d in random.sample(val_dataset_dicts, num_samples):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=val_dataset_metadata)
    outputs = predictor(img)
    v = Visualizer(img[:,:,::-1], val_dataset_metadata, scale=2)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

    print('Ground Truth')
    [print(pobject['bbox']) for pobject in d['annotations']]
    print('Prediction')
    [print(box) for box in pred_abox_list(outputs)]